In [397]:
from googleapiclient.discovery import build
from datetime import datetime
import pandas as pd
import numpy as np
import csv

In [398]:
api_key = 'put your api key here'
token_file = 'youtube_data/nextToken.txt'

In [399]:
with open(token_file, 'r') as tokenfile:
    page_token = tokenfile.read()

In [400]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [401]:
type(youtube)

googleapiclient.discovery.Resource

In [402]:
start_time = datetime(year=2017, month=11, day=14).strftime('%Y-%m-%dT%H:%M:%SZ')
end_time = datetime(year=2018, month=6, day=14).strftime('%Y-%m-%dT%H:%M:%SZ')

In [403]:
res = youtube.search().list(part='snippet',
                            maxResults=50,
                            publishedAfter=start_time,
                            publishedBefore=end_time,
                            order='viewCount',
                            type='video',
                            regionCode='GB',
                            pageToken=page_token
                           ).execute()

In [404]:
with open(token_file, 'w') as tokenfile:
    tokenfile.write(res['nextPageToken'])

In [405]:
vid_dict = {'videoId':[], 'publishedAt':[], 'title':[], 'description':[], 'thumbnail':[], 'channelTitle':[], 'liveBcast':[]}
for item in res['items']:
    vid_dict['videoId'].append(item['id']['videoId'])
    vid_dict['publishedAt'].append(item['snippet']['publishedAt'])
    vid_dict['title'].append(item['snippet']['title'])
    vid_dict['description'].append(item['snippet']['description'])
    vid_dict['thumbnail'].append(item['snippet']['thumbnails']['default']['url'])
    vid_dict['channelTitle'].append(item['snippet']['channelTitle'])
    vid_dict['liveBcast'].append(item['snippet']['liveBroadcastContent'])

In [406]:
videos = pd.DataFrame.from_dict(vid_dict)

In [407]:
now = datetime.now().strftime("%d.%m.%H.%M.%S")

In [408]:
videos.to_csv("./youtube_data/nontrending" + now + page_token + ".csv", index=False, sep=';')